In [1]:
from raise_utils.learners import FeedforwardDL, Learner
from raise_utils.transform import Transform
from typing import Callable
from raise_utils.data import DataLoader, Data
from raise_utils.metrics import ClassificationMetrics
import random
import copy
import time

In [2]:
class SteppedSearch:
    def __init__(self, params: dict, metrics: list, data: Data, n_iters: int, model_fn: Callable):
        """
        Initializes the hyper-parameter optimizer.
        
        :param params: dict - dict of hyper-parameters to tune. The schema is as follows:
        {
            "param_name": {
                "start": Any,
                "range": Union[list, tuple],
                "step_size": Union[int, float],
                "is_learner_param": bool                
            }
        }
        Please put the params with a list type of range at the end.
        """
        self.params = params
        self.metrics = metrics
        self.data = data
        self.n_iters = n_iters
        self.model_fn = model_fn
        self.cur_hp_dict = {k: v["start"] for k, v in self.params.items()}
        self.cache = {}
    
    def _get_model_performance(self, model: Learner, pre: str):
        cache_str = '|'.join([str(self.cur_hp_dict[x]) for x in self.cur_hp_dict.keys()])
        if cache_str in self.cache:
            return self.cache[cache_str]
        
        transform = Transform(pre)
        dt = copy.deepcopy(self.data)
        transform.apply(dt)
        model.set_data(*dt)
        model.fit()
        preds = model.predict(self.data.x_test)
        metr = ClassificationMetrics(self.data.y_test, preds)
        metr.add_metrics(self.metrics)
        perfs = metr.get_metrics()
        
        self.cache[cache_str] = perfs
        return perfs
        
    def _in_range(self, param, val):
        if isinstance(self.params[param]["range"], list):
            return True
        
        return self.params[param]["range"][0] <= val <= self.params[param]["range"][1]
    
    def _get_next_trend_step(self, cur_hp_dict: dict, cur_trend: dict):
        print(cur_hp_dict, cur_trend)
        if cur_trend:
            # Not our first run
            # What are we trying now?
            cur_param = list(cur_trend.keys())[0]
            
            if isinstance(self.params[cur_param]["range"], list):
                pass
            else:
                if self._in_range(cur_param, cur_trend[cur_param] + cur_trend["trend"] * self.params[cur_param]["step_size"]):
                    cur_trend[cur_param] += cur_trend["trend"] * self.params[cur_param]["step_size"]
                    return cur_trend
                
                # Try the next one
                idx = list(self.params.keys()).index(cur_param)
                if idx == len(self.params):
                    return False
                else:
                    cur_param = list(self.params.keys())[idx + 1]
                    cur_trend = {cur_param: self.params[cur_param]["start"], "trend": +1}
                    return cur_trend
        else:
            # First run            
            cur_param = list(self.params.keys())[0]
            cur_trend = {cur_param: self.params[cur_param]["start"] + self.params[cur_param]["step_size"], "trend": +1}
            return cur_trend
    
    def optimize(self):
        cur_trend = {}
        cur_best = -1.
        cur_best_perfs = []
        cur_best_config = copy.deepcopy(self.cur_hp_dict)
        i = 1
        
        while i <= self.n_iters:
            print('iter', i)
            i += 1
            # Get the model
            model = self.model_fn(self.cur_hp_dict)
            
            # Test it
            perfs = self._get_model_performance(model, self.cur_hp_dict["preprocessor"])
            
            # Did we improve?
            if perfs[0] > cur_best:
                # Save this config
                cur_best = perfs[0]
                cur_best_perfs = perfs
                cur_best_config = copy.deepcopy(self.cur_hp_dict)
                
                # Get our next update (else fail)
                cur_trend = self._get_next_trend_step(self.cur_hp_dict, cur_trend)
                if cur_trend == False:
                    print('Complete')
                    return cur_best_config, cur_best_perfs
                cur_param = list(cur_trend.keys())[0]
                self.cur_hp_dict[cur_param] = cur_trend[cur_param]
                # TODO: Set cur_hp_dict....probably needs rewriting since loop needed
            else:
                # Was this a choice or a range?
                cur_param = list(cur_trend.keys())[0]
                if isinstance(self.params[cur_param]["range"], list):
                    # As long as there's something to try
                    while self.params[cur_param]["range"].index(cur_trend[cur_param]) < len(self.params[cur_param]["range"]) - 1:
                        idx = self.params[cur_param]["range"].index(cur_trend[cur_param])
                        cur_param_val = self.params[cur_param]["range"][idx + 1]
                        tmp = copy.deepcopy(self.cur_hp_dict)
                        tmp[cur_param] = cur_param_val
                        
                        # Is it better?
                        perfs = self._get_model_performance(self.model_fn(tmp), tmp["preprocessor"])
                        if perfs[0] > cur_best:
                            # Save this config
                            cur_best = perfs[0]
                            cur_best_perfs = perfs
                            cur_best_config = copy.deepcopy(self.cur_hp_dict)
                            self.cur_hp_dict[cur_param] = cur_param_val
                            cur_trend[cur_param] = cur_param_val
                        else:
                            # Try the next one
                            cur_trend[cur_param] = cur_param_val
                    
                    # cur_best_config already reflects the best config for this
                    # param, so there's nothing to do here except move to the next param.
                    idx = list(self.params.keys()).index(cur_param) + 1
                    if idx == len(list(self.params.keys())):
                        # No more params to look at, we're done.
                        return cur_best_config, cur_best_perfs
                    else:
                        # Set the trend now
                        param = list(self.params.keys())[idx]
                        cur_trend = {param: self.params[param]["start"], "trend": self.params[param]["step_size"]}
                        continue
                else:
                    # Go the current way...
                    cur_trend = self._get_next_trend_step(self.cur_hp_dict, cur_trend)
                    if cur_trend == False:
                        print("Complete.")
                        return cur_best_config, cur_best_perfs
                    
                    tmp = copy.deepcopy(self.cur_hp_dict)
                    perfs = self._get_model_performance(self.model_fn(tmp), self.cur_hp_dict["preprocessor"])
                    
                    # Did it work?
                    perfs = self._get_model_performance(self.model_fn(tmp), self.cur_hp_dict["preprocessor"])
                    if perfs[0] > cur_best:
                        # Save this config
                        cur_best = perfs[0]
                        cur_best_perfs = perfs
                        cur_best_config = copy.deepcopy(self.cur_hp_dict)
                        cur_param = list(cur_trend.keys())[0]
                        self.cur_hp_dict[cur_param] = cur_trend[cur_param]
                    else:
                        # What about the other way?
                        cur_trend["trend"] = -cur_trend["trend"]
                        tmp = copy.deepcopy(self.cur_hp_dict)
                        perfs = self._get_model_performance(self.model_fn(tmp), self.cur_hp_dict["preprocessor"])

                        # Did that work?
                        if perfs[0] > cur_best:
                            # Save this config
                            cur_best = perfs[0]
                            cur_best_perfs = perfs
                            cur_best_config = copy.deepcopy(self.cur_hp_dict)
                            self.cur_hp_dict[cur_param] = cur_trend[cur_param]
                        else:
                            # This parameter is pointless. Reset and move on. We abuse
                            # the range validation in _get_next_trend_step to do this
                            # for us.
                            cur_trend[cur_param] = self.params[cur_param]["range"][1]
                            cur_trend = self._get_next_trend_step(self.cur_hp_dict, cur_trend)
                            if cur_trend == False:
                                print("Complete.")
                                return cur_best_config, cur_best_perfs

In [3]:
file_dic = {"ivy":     ["ivy-1.1.csv", "ivy-1.4.csv", "ivy-2.0.csv"],\
        "lucene":  ["lucene-2.0.csv", "lucene-2.2.csv", "lucene-2.4.csv"],\
        "poi":     ["poi-1.5.csv", "poi-2.0.csv", "poi-2.5.csv", "poi-3.0.csv"],\
        "synapse": ["synapse-1.0.csv", "synapse-1.1.csv", "synapse-1.2.csv"],\
        "velocity":["velocity-1.4.csv", "velocity-1.5.csv", "velocity-1.6.csv"], \
        "camel": ["camel-1.0.csv", "camel-1.2.csv", "camel-1.4.csv", "camel-1.6.csv"], \
        "jedit": ["jedit-3.2.csv", "jedit-4.0.csv", "jedit-4.1.csv", "jedit-4.2.csv", "jedit-4.3.csv"], \
        "log4j": ["log4j-1.0.csv", "log4j-1.1.csv", "log4j-1.2.csv"], \
        "xalan": ["xalan-2.4.csv", "xalan-2.5.csv", "xalan-2.6.csv", "xalan-2.7.csv"], \
        "xerces": ["xerces-1.2.csv", "xerces-1.3.csv", "xerces-1.4.csv"]
        }

In [4]:
metrics = ["d2h2", "pd", "pf"]

In [5]:
hyp = {
    "n_layers": {
        "start": 1,
        "range": (1, 5),
        "step_size": 1,
        "is_learner_param": True
    },
    "n_units": {
        "start": 3,
        "range": (3, 20),
        "step_size": 2,
        "is_learner_param": True
    },
    "preprocessor": {
        "start": "normalize",
        "range": ["normalize", "standardize", "minmax", "maxabs"],
        "step_size": 1,
        "is_learner_param": False
    }
}

In [6]:
def model_fn(config):
    """
    Returns a deep learner given the current config.
    """
    learner = FeedforwardDL(weighted=True, wfo=True, n_layers=config["n_layers"], 
                            n_units=config["n_units"], n_epochs=30)
    return learner

In [4]:
data = DataLoader.from_files(base_path='../Dodge/data/defect/', files=file_dic["ivy"])

In [93]:
opt = SteppedSearch(params=hyp, metrics=metrics, data=data, model_fn=model_fn, n_iters=30)

In [94]:
opt.optimize()

iter 1
got model
Epoch 1/30
75/75 [==============================] - 0s 770us/step - loss: 1.9016
Epoch 2/30
75/75 [==============================] - 0s 649us/step - loss: 1.8776
Epoch 3/30
75/75 [==============================] - 0s 620us/step - loss: 1.8683
Epoch 4/30
75/75 [==============================] - 0s 628us/step - loss: 1.8611
Epoch 5/30
75/75 [==============================] - 0s 647us/step - loss: 1.8550
Epoch 6/30
75/75 [==============================] - 0s 1ms/step - loss: 1.8478
Epoch 7/30
75/75 [==============================] - 0s 959us/step - loss: 1.8409
Epoch 8/30
75/75 [==============================] - 0s 858us/step - loss: 1.8340
Epoch 9/30
75/75 [==============================] - 0s 594us/step - loss: 1.8266
Epoch 10/30
75/75 [==============================] - 0s 525us/step - loss: 1.8192
Epoch 11/30
75/75 [==============================] - 0s 592us/step - loss: 1.8125
Epoch 12/30
75/75 [==============================] - 0s 530us/step - loss: 1.8044
Epoch 13/3

({'n_layers': 1, 'n_units': 3, 'preprocessor': 'normalize'},
 [0.36874602501803966, 0.9, 0.46794871794871795])

In [7]:
for dataset in file_dic.keys():
    print(dataset)
    print('=' * len(dataset))
    data = DataLoader.from_files(base_path='../Dodge/data/defect/', files=file_dic[dataset])
    opt = SteppedSearch(params=hyp, metrics=metrics, data=data, model_fn=model_fn, n_iters=30)
    start = time.time()
    config, perfs = opt.optimize()
    end = time.time()
    print('Time taken:', round(end - start, 2), 'seconds.')
    print(perfs)

ivy
===
iter 1
Epoch 1/30
75/75 [==============================] - 0s 607us/step - loss: 1.8909
Epoch 2/30
75/75 [==============================] - 0s 538us/step - loss: 1.8904
Epoch 3/30
75/75 [==============================] - 0s 525us/step - loss: 1.8871
Epoch 4/30
75/75 [==============================] - 0s 578us/step - loss: 1.8775
Epoch 5/30
75/75 [==============================] - 0s 515us/step - loss: 1.8674
Epoch 6/30
75/75 [==============================] - 0s 576us/step - loss: 1.8565
Epoch 7/30
75/75 [==============================] - 0s 521us/step - loss: 1.8452
Epoch 8/30
75/75 [==============================] - 0s 626us/step - loss: 1.8340
Epoch 9/30
75/75 [==============================] - 0s 611us/step - loss: 1.8237
Epoch 10/30
75/75 [==============================] - 0s 571us/step - loss: 1.8137
Epoch 11/30
75/75 [==============================] - 0s 544us/step - loss: 1.8058
Epoch 12/30
75/75 [==============================] - 0s 621us/step - loss: 1.7984
Epoch 13/3

Time taken: 6.62 seconds.
[0.3897129829757613, 0.65, 0.1987179487179487]
lucene
iter 1
Epoch 1/30
45/45 [==============================] - 0s 732us/step - loss: 0.9932
Epoch 2/30
45/45 [==============================] - 0s 675us/step - loss: 0.9909
Epoch 3/30
45/45 [==============================] - 0s 795us/step - loss: 0.9889
Epoch 4/30
45/45 [==============================] - 0s 684us/step - loss: 0.9872
Epoch 5/30
45/45 [==============================] - 0s 674us/step - loss: 0.9857
Epoch 6/30
45/45 [==============================] - 0s 691us/step - loss: 0.9845
Epoch 7/30
45/45 [==============================] - 0s 668us/step - loss: 0.9832
Epoch 8/30
45/45 [==============================] - 0s 715us/step - loss: 0.9822
Epoch 9/30
45/45 [==============================] - 0s 709us/step - loss: 0.9811
Epoch 10/30
45/45 [==============================] - 0s 857us/step - loss: 0.9803
Epoch 11/30
45/45 [==============================] - 0s 773us/step - loss: 0.9791
Epoch 12/30
45/45 [=

187/187 [==============================] - 0s 537us/step - loss: 0.9928
Epoch 29/30
187/187 [==============================] - 0s 505us/step - loss: 0.9925
Epoch 30/30
187/187 [==============================] - 0s 546us/step - loss: 0.9916
{'n_layers': 1, 'n_units': 3, 'preprocessor': 'normalize'} {}
iter 2
Epoch 1/30
187/187 [==============================] - 0s 611us/step - loss: 1.1194
Epoch 2/30
187/187 [==============================] - 0s 561us/step - loss: 1.0779
Epoch 3/30
187/187 [==============================] - 0s 609us/step - loss: 1.0518
Epoch 4/30
187/187 [==============================] - 0s 600us/step - loss: 1.0312
Epoch 5/30
187/187 [==============================] - 0s 591us/step - loss: 1.0160
Epoch 6/30
187/187 [==============================] - 0s 618us/step - loss: 1.0048
Epoch 7/30
187/187 [==============================] - 0s 605us/step - loss: 0.9960
Epoch 8/30
187/187 [==============================] - 0s 678us/step - loss: 0.9880
Epoch 9/30
187/187 [=======

72/72 [==============================] - 0s 482us/step - loss: 1.9016
Epoch 26/30
72/72 [==============================] - 0s 527us/step - loss: 1.8941
Epoch 27/30
72/72 [==============================] - 0s 476us/step - loss: 1.8895
Epoch 28/30
72/72 [==============================] - 0s 479us/step - loss: 1.8825
Epoch 29/30
72/72 [==============================] - 0s 472us/step - loss: 1.8775
Epoch 30/30
72/72 [==============================] - 0s 479us/step - loss: 1.8708
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_layers': 2, 'trend': 1}
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_layers': 5, 'trend': -1}
iter 3
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_layers': 4, 'trend': -1}
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_layers': 5, 'trend': 1}
iter 4
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_units': 3, 'trend': 1}
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_units':

55/55 [==============================] - 0s 615us/step - loss: 0.7741
Epoch 24/30
55/55 [==============================] - 0s 628us/step - loss: 0.7724
Epoch 25/30
55/55 [==============================] - 0s 647us/step - loss: 0.7705
Epoch 26/30
55/55 [==============================] - 0s 643us/step - loss: 0.7692
Epoch 27/30
55/55 [==============================] - 0s 566us/step - loss: 0.7679
Epoch 28/30
55/55 [==============================] - 0s 570us/step - loss: 0.7662
Epoch 29/30
55/55 [==============================] - 0s 556us/step - loss: 0.7648
Epoch 30/30
55/55 [==============================] - 0s 590us/step - loss: 0.7632
{'n_layers': 3, 'n_units': 3, 'preprocessor': 'normalize'} {'n_layers': 3, 'trend': 1}
{'n_layers': 3, 'n_units': 3, 'preprocessor': 'normalize'} {'n_layers': 5, 'trend': -1}
iter 4
{'n_layers': 3, 'n_units': 3, 'preprocessor': 'normalize'} {'n_layers': 4, 'trend': -1}
{'n_layers': 3, 'n_units': 3, 'preprocessor': 'normalize'} {'n_layers': 5, 'trend': 1}

274/274 [==============================] - 0s 597us/step - loss: 1.5916
Epoch 11/30
274/274 [==============================] - 0s 568us/step - loss: 1.5849
Epoch 12/30
274/274 [==============================] - 0s 559us/step - loss: 1.5797
Epoch 13/30
274/274 [==============================] - 0s 613us/step - loss: 1.5750
Epoch 14/30
274/274 [==============================] - 0s 559us/step - loss: 1.5714
Epoch 15/30
274/274 [==============================] - 0s 593us/step - loss: 1.5680
Epoch 16/30
274/274 [==============================] - 0s 593us/step - loss: 1.5650
Epoch 17/30
274/274 [==============================] - 0s 585us/step - loss: 1.5622
Epoch 18/30
274/274 [==============================] - 0s 536us/step - loss: 1.5597
Epoch 19/30
274/274 [==============================] - 0s 625us/step - loss: 1.5578
Epoch 20/30
274/274 [==============================] - 0s 596us/step - loss: 1.5556
Epoch 21/30
274/274 [==============================] - 0s 600us/step - loss: 1.5535
Epoc

40/40 [==============================] - 0s 575us/step - loss: 1.5306
Epoch 8/30
40/40 [==============================] - 0s 673us/step - loss: 1.5280
Epoch 9/30
40/40 [==============================] - 0s 719us/step - loss: 1.5244
Epoch 10/30
40/40 [==============================] - 0s 846us/step - loss: 1.5192
Epoch 11/30
40/40 [==============================] - 0s 864us/step - loss: 1.5128
Epoch 12/30
40/40 [==============================] - 0s 763us/step - loss: 1.5032
Epoch 13/30
40/40 [==============================] - 0s 792us/step - loss: 1.4906
Epoch 14/30
40/40 [==============================] - 0s 944us/step - loss: 1.4759
Epoch 15/30
40/40 [==============================] - 0s 1ms/step - loss: 1.4606
Epoch 16/30
40/40 [==============================] - 0s 871us/step - loss: 1.4429
Epoch 17/30
40/40 [==============================] - 0s 663us/step - loss: 1.4247
Epoch 18/30
40/40 [==============================] - 0s 660us/step - loss: 1.4072
Epoch 19/30
40/40 [=============

398/398 [==============================] - 0s 561us/step - loss: 1.1537
Epoch 5/30
398/398 [==============================] - 0s 619us/step - loss: 1.1480
Epoch 6/30
398/398 [==============================] - 0s 572us/step - loss: 1.1424
Epoch 7/30
398/398 [==============================] - 0s 606us/step - loss: 1.1385
Epoch 8/30
398/398 [==============================] - 0s 581us/step - loss: 1.1341
Epoch 9/30
398/398 [==============================] - 0s 686us/step - loss: 1.1300
Epoch 10/30
398/398 [==============================] - 0s 632us/step - loss: 1.1236
Epoch 11/30
398/398 [==============================] - 0s 609us/step - loss: 1.1147
Epoch 12/30
398/398 [==============================] - 0s 645us/step - loss: 1.1022
Epoch 13/30
398/398 [==============================] - 0s 652us/step - loss: 1.0851
Epoch 14/30
398/398 [==============================] - 0s 675us/step - loss: 1.0679
Epoch 15/30
398/398 [==============================] - 0s 657us/step - loss: 1.0520
Epoch 16/

{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_layers': 4, 'trend': -1}
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_layers': 5, 'trend': 1}
iter 4
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_units': 3, 'trend': 1}
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_units': 20, 'trend': -1}
iter 5
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_units': 18, 'trend': -1}
{'n_layers': 2, 'n_units': 3, 'preprocessor': 'normalize'} {'n_units': 20, 'trend': 1}
iter 6
Time taken: 8.15 seconds.
[0.060454078274451484, 0.10297482837528604, 0.12582781456953643]
